In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Chromophobe/GSE68606'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the description, the dataset involves gene expression analysis

# Find the row keys for the variables
trait_row = 7 if 'Kidney_Chromophobe' in [val.split(': ')[-1] for val in {key: lst[-1] for key, lst in {7: ['histology: Recurrent Renal Cell Carcinoma, chromophobe cell type']}.items()}[7]] else None
age_row = 6 if len(set([val.split(': ')[-1] for val in {key: lst for key, lst in {6: ['age: 67', 'age: 66', 'age: 72', 'age: 56', 'age: 48']}.items()}[6]])) > 1 else None
gender_row = 5 if len(set([val.split(': ')[-1] for val in {key: lst for key, lst in {5: ['Sex: female', 'Sex: male']}.items()}[5]])) > 1 else None

# Define data conversion functions
def convert_trait(value):
    if value == 'Recurrent Renal Cell Carcinoma, chromophobe cell type':
        return 1
    if value:
        return 0
    return None

def convert_age(value):
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    if value == 'female':
        return 0
    if value == 'male':
        return 1
    return None

# Save the cohort information
save_cohort_info('GSE68606', './preprocessed/Kidney_Chromophobe/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Chromophobe', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Chromophobe/trait_data/GSE68606.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
